In [1]:
import json
import re

In [51]:
email_batches = json.load(open("../artifacts/emails_batch2.json"))["batch"]

In [53]:
len(email_batches)

4

In [54]:
import tiktoken

def num_tokens_from_text(text, model="gpt-4"):
    encoding = tiktoken.encoding_for_model(model)
    tokens = encoding.encode(text)
    return len(tokens)

# Example usage
text = "Hello, how are you today?"
token_count = num_tokens_from_text(text)
print(f"Token count: {token_count}")

Token count: 7


In [55]:
for batch in email_batches:
    print(type(json.dumps(batch)))
    break

<class 'str'>


In [58]:
token_lens = []
for batch_idx, batch in enumerate(email_batches):
    token_len = num_tokens_from_text(json.dumps(batch))
    token_lens.append((token_len, batch_idx))
    print(token_len)

1104
1196
805
1013


In [57]:
print(max(token_lens))

(1196, 1)


In [76]:
num_tokens_from_text("hello"+"\n"*12+"world")

3

In [8]:
email_batches[11]

{'batch': [{'type': 'received',
   'subject': 'Red Rooster Coffee, your performance for April 2025',
   'from': 'googlebase-noreply@google.com',
   'to': 'matt@bluewatermarketing.com',
   'date': '2025-05-08T16:03:49.000Z',
   'content': "\n\n\n\n\n\n\n\n\n\nYour customer ID: 603232330\n\nYour business performance for April 2025\n\n\n\n\n\n\n\nHow your products performed (ads only)\nClicks\n282\n-51.3%*\n\nImpressions\n62.62K\n-45.5%*\n\nCTR\n0.5%\n-10.6%*\n\n\nManage campaigns <https://notifications.google.com/g/p/ANiao5p3ia-SeXXH6DD5_o9EMiDpsTG8_4IeiuiwDqjR4G97cJ1QmolP-oxN4lQnttX_GJClvGaihKmwXSPSgcsdmFSRjr5V9gR6N2uN0SYAMeJo13DlTRFPFMNZ9EaznFJKFF4d1-Yq-4cw8eA9vz-D51o8oPc-o6OEoALhh72JjXUnudKmChC6phKod1Db5T-hSUVpCeyRHn_r_4NKAhJ1BuVJb8oayLqOOA7QwMrrhPopTpK5DNJFqXuLZzkNlwM-XHPPixGymcYaDEMHPAp7JZnS3hIb90wnWlisxB_Oe-xVzd4DYaNNcfBjNck>\n\n\n\n\n\n\nHow your products performed (organic & ads)\nClicks\n670\n-31.4%*\n\nImpressions\n89.26K\n-34.1%*\n\nCTR\n0.8%\n+4.1%*\n\n\n\n\n*Compared to Marc

In [40]:
for email in email_batches[11]["batch"]:
    print(len(email["content"]))
    print(email)
    break

4831
{'type': 'received', 'subject': 'Red Rooster Coffee, your performance for April 2025', 'from': 'googlebase-noreply@google.com', 'to': 'matt@bluewatermarketing.com', 'date': '2025-05-08T16:03:49.000Z', 'content': "\n\n\n\n\n\n\n\n\n\nYour customer ID: 603232330\n\nYour business performance for April 2025\n\n\n\n\n\n\n\nHow your products performed (ads only)\nClicks\n282\n-51.3%*\n\nImpressions\n62.62K\n-45.5%*\n\nCTR\n0.5%\n-10.6%*\n\n\nManage campaigns <https://notifications.google.com/g/p/ANiao5p3ia-SeXXH6DD5_o9EMiDpsTG8_4IeiuiwDqjR4G97cJ1QmolP-oxN4lQnttX_GJClvGaihKmwXSPSgcsdmFSRjr5V9gR6N2uN0SYAMeJo13DlTRFPFMNZ9EaznFJKFF4d1-Yq-4cw8eA9vz-D51o8oPc-o6OEoALhh72JjXUnudKmChC6phKod1Db5T-hSUVpCeyRHn_r_4NKAhJ1BuVJb8oayLqOOA7QwMrrhPopTpK5DNJFqXuLZzkNlwM-XHPPixGymcYaDEMHPAp7JZnS3hIb90wnWlisxB_Oe-xVzd4DYaNNcfBjNck>\n\n\n\n\n\n\nHow your products performed (organic & ads)\nClicks\n670\n-31.4%*\n\nImpressions\n89.26K\n-34.1%*\n\nCTR\n0.8%\n+4.1%*\n\n\n\n\n*Compared to March 2025\n\n\nYour popu

In [9]:
num_tokens_from_text(json.dumps(email_batches[11]["batch"][1]))

9098

In [10]:
content_len=4831
token_len=2492

In [11]:
import re

text = r'work\\n\\\\\\\\\\n\\\\\\\\\\n\\\\\\\\\\n\\\\\\\\\\n\\\\\\\\\\n\\\\\\\\\\n\\\\\\\\\\n\\\\\\\\\\n\\\\\\\\\\n\\\\\\\\\\n\\\\\\\\\\n\\\\\\\\\\n\\\\\\nCanva'
def clean_text(text):
    # Step 1: Replace all sequences of backslashes followed by n with a single \n
    cleaned_text = re.sub(r'\\+n', r'\n', text)

    # Step 2: Remove any remaining standalone backslashes
    cleaned_text = re.sub(r'\\+', '', cleaned_text)

    # Step 3: Replace multiple newlines with a single newline
    cleaned_text = re.sub(r'\n+', r'\n', cleaned_text)

    return cleaned_text

print(clean_text(text))

work
Canva


In [12]:
import re
import codecs

def clean_email_text(text):
    # 1. Decode escaped unicode (e.g., \\u034f -> \u034f)
    try:
        text = codecs.decode(text, 'unicode_escape')
    except Exception:
        pass

    # 2. Remove invisible/control characters
    text = re.sub(r'[\u200c\u200b\u200d\u034f\u202c\u202d\u00a0\ufeff]+', '', text)

    # 3. Remove URLs (http, https, www)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)

    # 4. Collapse multiple newlines, even with junk/invisible chars in between
    text = re.sub(r'(\n[\s\u200c\u034f\ufeff\u00a0]*){2,}', '\n', text)

    # 5. Normalize internal whitespace and strip leading/trailing spaces
    text = re.sub(r'[ \t]+', ' ', text)
    text = re.sub(r' *\n *', '\n', text).strip()

    return text


In [13]:
i = 1
content = email_batches[11]["batch"][i]["content"]
cleaned_content = clean_email_text(json.dumps(content))
content_len = len(cleaned_content)
token_len = num_tokens_from_text(json.dumps(cleaned_content))
print(f"{content_len=}\n{token_len=}")
print()
print(cleaned_content)

content_len=1205
token_len=356

"Get inspired by real people using Canva at work
Canva
[
Design with Canva
[
[
[
MonthlyGlowUp
How Erin transforms ideas into organized plans
[
This month, let Erin show you how to go from vision to execution in less than 5
minutes. Check out our Top 4 AI Hacks
[
and watch how she transforms notes from a brainstorming session into an
organized action plan with just a few clicks.
Watch the video
[
[
HOW SHE DOES IT (AND HOW YOU CAN TOO)
[
[
SORT NOTES WITH ONE CLICK
Organize sticky notes by name, topic, color, or reactions for instant clarity
[
SUMMARIZE MESSY BRAINSTORMS
Quickly identify key points in your whiteboard with Magic Write
[
CONDENSE IDEAS INTO A DOC
Use Magic Switch to turn your whiteboard into a rollout plan in seconds
Try it now
[
You are receiving this email because you signed up to Canva.
Get Canva for:
iPhone
[
iPad
[
Android
[
Mac
[
Windows
[
Download the Canva app
app store
[
google play
[
This advertisement is made for you withlove
[


In [14]:
import pyperclip

In [15]:
pyperclip.copy(email_batches[11]["batch"][i])

In [18]:
cleaned_content = [
    {
        "type": "received",
        "subject": "Monthly Glow Up: Organize your ideas in just 5 minutes",
        "from": "marketing@engage.canva.com",
        "to": "matt@bluewatermarketing.com",
        "date": "2025-05-08T15:06:11.000Z",
        "content": "Get inspired by real people using Canva at work\nCanva\n[\nDesign with Canva\n[\n[\n[\nMonthlyGlowUp\nHow Erin transforms ideas into organized plans\n[\nThis month, let Erin show you how to go from vision to execution in less than 5\nminutes. Check out our Top 4 AI Hacks\n[\nand watch how she transforms notes from a brainstorming session into an\norganized action plan with just a few clicks.\nWatch the video\n[\n[\nHOW SHE DOES IT (AND HOW YOU CAN TOO)\n[\n[\nSORT NOTES WITH ONE CLICK\nOrganize sticky notes by name, topic, color, or reactions for instant clarity\n[\nSUMMARIZE MESSY BRAINSTORMS\nQuickly identify key points in your whiteboard with Magic Write\n[\nCONDENSE IDEAS INTO A DOC\nUse Magic Switch to turn your whiteboard into a rollout plan in seconds\nTry it now\n[\nYou are receiving this email because you signed up to Canva.\nGet Canva for:\niPhone\n[\niPad\n[\nAndroid\n[\nMac\n[\nWindows\n[\nDownload the Canva app\napp store\n[\ngoogle play\n[\nThis advertisement is made for you withlove\n[\nCanva\nCanva Pty Ltd, 110 Kippax St, NSW 2010, Australia. ABN 80 158 929 938\nIf you need assistance, get in touch with us via our Help Center\n[\nVisit canva.com\n[\nUnsubscribe\n[\nMessage Preferences\n[\nfacebook\n[\nx\n[\npinterest\n[\ninstagram\n[\n["
    }
]

In [20]:
print(cleaned_content[0]["content"])

Get inspired by real people using Canva at work
Canva
[
Design with Canva
[
[
[
MonthlyGlowUp
How Erin transforms ideas into organized plans
[
This month, let Erin show you how to go from vision to execution in less than 5
minutes. Check out our Top 4 AI Hacks
[
and watch how she transforms notes from a brainstorming session into an
organized action plan with just a few clicks.
Watch the video
[
[
HOW SHE DOES IT (AND HOW YOU CAN TOO)
[
[
SORT NOTES WITH ONE CLICK
Organize sticky notes by name, topic, color, or reactions for instant clarity
[
SUMMARIZE MESSY BRAINSTORMS
Quickly identify key points in your whiteboard with Magic Write
[
CONDENSE IDEAS INTO A DOC
Use Magic Switch to turn your whiteboard into a rollout plan in seconds
Try it now
[
You are receiving this email because you signed up to Canva.
Get Canva for:
iPhone
[
iPad
[
Android
[
Mac
[
Windows
[
Download the Canva app
app store
[
google play
[
This advertisement is made for you withlove
[
Canva
Canva Pty Ltd, 110 Kippax S

In [26]:
received_emails = json.load(open("../artifacts/received_emails2.json"))

In [41]:
for i, email in enumerate(received_emails):
    if email["subject"] == "Red Rooster Coffee, your performance for April 2025":
        print(i)
        break

44


In [43]:
email

{'id': '196b0a3b9709100c',
 'threadId': '196b0a3b9709100c',
 'labelIds': ['Label_7954421186598713333', 'CATEGORY_UPDATES', 'INBOX'],
 'sizeEstimate': 87006,
 'headers': {'delivered-to': 'Delivered-To: matt@bluewatermarketing.com',
  'received': 'Received: from mail-sor-f69.google.com (mail-sor-f69.google.com. [209.85.220.69])\r\n        by mx.google.com with SMTPS id 5614622812f47-403484c7222sor1629207b6e.15.2025.05.08.09.04.01\r\n        for <matt@bluewatermarketing.com>\r\n        (Google Transport Security);\r\n        Thu, 08 May 2025 09:04:01 -0700 (PDT)',
  'x-received': 'X-Received: by 2002:a05:6214:f28:b0:6e5:bc9:95f8 with SMTP id\r\n 6a1803df08f44-6f542a1988bmr135648086d6.17.1746720229952; Thu, 08 May 2025\r\n 09:03:49 -0700 (PDT)',
  'arc-seal': 'ARC-Seal: i=1; a=rsa-sha256; t=1746720241; cv=none;\r\n        d=google.com; s=arc-20240605;\r\n        b=EXDoVaYq1FKRwUvEuT7IpJ1fx7M0RN+3UNmjYwwULEvffnL4GKrS0UcMsFBj0qOMev\r\n         3LuWsAkKs+r2hiNU3YGV7sradhKbZz/DJyQFv8rLufYNyZ4i

In [45]:
print(email["text"])











Your customer ID: 603232330

Your business performance for April 2025







How your products performed (ads only)
Clicks
282
-51.3%*

Impressions
62.62K
-45.5%*

CTR
0.5%
-10.6%*


Manage campaigns <https://notifications.google.com/g/p/ANiao5p3ia-SeXXH6DD5_o9EMiDpsTG8_4IeiuiwDqjR4G97cJ1QmolP-oxN4lQnttX_GJClvGaihKmwXSPSgcsdmFSRjr5V9gR6N2uN0SYAMeJo13DlTRFPFMNZ9EaznFJKFF4d1-Yq-4cw8eA9vz-D51o8oPc-o6OEoALhh72JjXUnudKmChC6phKod1Db5T-hSUVpCeyRHn_r_4NKAhJ1BuVJb8oayLqOOA7QwMrrhPopTpK5DNJFqXuLZzkNlwM-XHPPixGymcYaDEMHPAp7JZnS3hIb90wnWlisxB_Oe-xVzd4DYaNNcfBjNck>






How your products performed (organic & ads)
Clicks
670
-31.4%*

Impressions
89.26K
-34.1%*

CTR
0.8%
+4.1%*




*Compared to March 2025


Your popular products
Red Rooster Ethiopia Worka Sakaro - Anaerobic Natural Berry & Floral Whole Bean (12OZ)
130 clicks

Red Rooster Organic Floyd Farmhouse Breakfast Blend - Almond & Cacao Medium Roast Whole Bean (12OZ)
29 clicks

Ethiopia Worka Sakaro Anaerobic Natural 2LB / French Pr

In [79]:
morning_brief = json.load(open("../artifacts/morning_brief.json"))

In [82]:
print(morning_brief[0]["message"]["content"])

Good morning,

Here is your brief for today:

Top 5 Action Items
1. Review and approve the 30-second Popcorn video for Bulk Candy Store (Rationale: Pending tasks from Billo App).
2. Review and respond to your Unread message on Upwork from Dinesh K (Rationale: Relevant to job DTC Static Ads Designer).
3. Review and approve Fred from Fireflies AI's meeting preparation and action items on 'PerfSol / Blue Water' project (Rationale: Necessary for project progress).
4. Respond to Ray from Atria about collaboration on case study spotlighting AI-powered ad optimization success (Rationale: Potential for extra visibility, credibility, and marketing materials for company).
5. Respond to Akshat Biyani's White Label partnership proposal for flexibility in pricing (Rationale: Potential beneficial partnership for Blue Water Marketing).

Urgent Follow-Ups
1. Confirm receipt of the updated list for May 2025 from Suparna Chakrobarty.
2. Confirm the acceptance of the strategy call with Phillip from ALWRL